In [5]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import tsplib95

def create_data_model(file_path):
    # Lê o arquivo .tsp
    problem = tsplib95.load(file_path)

    # Determina o número de nós e cria a matriz de distância
    nodes = list(problem.get_nodes())
    n = len(nodes)
    distance_matrix = [[0 for _ in range(n)] for _ in range(n)]

    for i in range(n):
        for j in range(n):
            if i != j:
                # Ajusta os índices para a matriz baseada em 0
                distance_matrix[i][j] = problem.get_weight(i+1, j+1)
            else:
                distance_matrix[i][j] = 0

    data = {}
    data['distance_matrix'] = distance_matrix
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data


def main():
    # Carrega os dados do problema
    data = create_data_model('test_bnb/test10.tsp')

    # Cria o modelo de roteamento
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)

    # Cria e registra uma função de custo de trânsito
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Resolve o problema
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    solution = routing.SolveWithParameters(search_parameters)

    # Imprime a solução
    if solution:
        print_solution(manager, routing, solution)

def print_solution(manager, routing, solution):
    # Função para imprimir a solução
    index = routing.Start(0)
    plan_output = 'Rota:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        # Ajusta o índice para mapear de volta para o esquema de indexação do arquivo .tsp
        plan_output += ' {} ->'.format(manager.IndexToNode(index) + 1)
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index) + 1)
    print(plan_output)
    print('Distância da rota: {} metros\n'.format(route_distance))

if __name__ == '__main__':
    main()


Rota:
 1 -> 5 -> 6 -> 4 -> 10 -> 9 -> 8 -> 3 -> 7 -> 2 -> 1

Distância da rota: 2826 metros



In [23]:
problem = tsplib95.load('test_bnb/test10.tsp')

In [24]:
# visualizando as arestas
print(list(problem.get_edges()))

[(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7), (7, 8), (7, 9), (7, 10), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (8, 10), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9), (9, 10), (10, 1), (10, 2), (10, 3), (10, 4), (10, 5), (10, 6), (10, 7), (10, 8), (10, 9), (10, 10)]


In [25]:
# peso entre aresta 1 e 2
print(problem.get_weight(1,2))

666
